# Фильмы с 1980 по 2020 годы

In [ ]:
import numpy as np #математические операции
import pandas as pd #функции для датасета
import matplotlib.pyplot as plt 
import scipy as sc #математический анализ
from numpy.random import randn
import seaborn as sns #статистические графики
from sklearn.impute import SimpleImputer
from scipy import stats

# 1. Загрузка данных
Для загрузки .csv файла с данными в pandas используется функция read_csv(). Выбираем директорию датасета Movies. 

In [ ]:
df= pd.read_csv ("D:\download\movie\Movies20.csv")

# 2. Информация о датасете 
Посмотрим информацию о датасете. Заголовки начало:

In [ ]:
df.head(10)

Заголовки конец:

In [ ]:
df.tail(10)

Записи и переменные:

In [ ]:
df.shape

Колонки:

In [ ]:
df.columns

Нулевые значения и типы данных.

In [ ]:
df.info()

Всего в таблице 15 столбцов, у 5 столбцов тип данных "float", у 1-"int" и у 9-"object".

Статистика числовых переменных

In [ ]:
df.describe()

# 3. Графики

Целевая переменная-budget, отвечающая за бюджет фильма. 
В первой строке остаются все переменные, кроме целевой. Во второй-только целевая.
Копия создается, чтобы изменение датасета не приводило к потере данных.

In [ ]:
X_df = df.drop(labels=['budget'], axis=1).copy()
y_df = df['budget'].values.copy()

Разделение данных

In [ ]:
X_df.shape, y_df.shape

Характеристики переменных. hist-cтроит гистограмму для каждой переменной

In [ ]:
X_df.shape, y_df.shape
df.hist(figsize=(9,9), bins=50, xlabelsize=8, ylabelsize=8);

Графики, подозрительные на выбросы(обладают нехарактерными всплесками).
Выбросы-точки, которые значительно выбиваются из общей "кучи". Подозрительным показалcя график "runtime".

In [ ]:
columns = ['runtime']
plt.figure(figsize=(18,8))
for i, col in enumerate(columns):
    tmp_ax = plt.subplot(3, 4, i+1)
    if not(i & 4 == 1):
        tmp_ax.label_outer()
df.plot(x = col, y='budget', ax=tmp_ax, kind='scatter')

Строим график Runtime в большем масштабе. Как видно, выбросы начинаются с отметки 160.

In [ ]:
fig, ax = plt.subplots( figsize=(9,5))
ax.scatter(x = df['runtime'], y = df['budget'])
plt.ylabel('budget', fontsize=13)
plt.xlabel('runtime', fontsize=13)
plt.show();

Выбросы можно выкинуть из данных или заменить медианными значениями. В данном случае, выбросы убираются.
Начинаются они с отметки 160 в runtime. Функция drop отбрасывает данные, которые не подходят под выражение.

In [ ]:
df = df.drop(df[(df['runtime']>160) & (df['budget']>4)].index)
fig, ax = plt.subplots()
ax.scatter(df['runtime'], df['budget'])
plt.ylabel('budget', fontsize=13)
plt.xlabel('runtime', fontsize=13)
plt.show()

Выбросы можно увидеть на графике

In [ ]:
data = pd.concat([df['budget'], df['year']], axis=1)
f, ax = plt.subplots(figsize=(22, 8))
fig = sns.boxplot(x='year', y="budget", data=data)

Попарные зависимости одной величины от другой. Каждая из переменных является основой одной из осей двумерного точечного графика.

In [ ]:
columns = ['year','budget','runtime','gross','score','votes'] #выбираем все колонки с численными значениями
pd.plotting.scatter_matrix(df[columns],figsize=(12,12));
plt.figure();

# 4. Анализ поисковых данных

Матрица корреляции. Строится с помощью тепловой карты и библиотеки seaborn. Темные поля-пропуски. Диагональные ячейки не рассматриваются, так как являются результатом расчета коэффициента между переменной и ее копией.

In [ ]:
corrmat = df.corr()
plt.subplots(figsize=(12,9))
sns.heatmap(corrmat, vmax=0.9, annot=True)

Матрица корреляции ниже имеет более удобный вариант, так как данные можно отфильтровать. К примеру, можно выбрать только те данные, где зависимость выше 0,4.

In [ ]:
plt.subplots(figsize=(9,9))
sns.heatmap(corrmat[(corrmat >= 0.4) | (corrmat < -0.4)], 
            cmap='viridis',
            vmax=1.0, vmin=-1.0,
            linewidth=0.1,
            annot=True,
            annot_kws={"size":8});

# 5. Пропуски

Проверим датасет на пропуски.Самое большое количество пропусков в колонках "бюджет", "gross", "rating".
Метод isnull() проходится по каждой ячейке каждого столбца и определяет, какой из них пустой, а какой нет.

In [ ]:
df.isnull().mean() * 100

Также пропуски можно увидеть в таблице. Видно,что они в 11 колонках. NaN -упрощенно "не число"

In [ ]:
df[df.columns[pd.isnull(df).sum() > 0]]

Пропуски можно удалить функцией df = df.drop(columns=['']) или заменить значениями. С помощью sklearn.impute.SimpleImputer вместо пропуска можно поставить значение. Было решено заполнить сразу все пропуски самыми распространенными значениями (most frequent).

In [ ]:
imputer = SimpleImputer(strategy='most_frequent', 
                        missing_values=np.nan)
imputer = imputer.fit(df)
df.iloc[:,:] = imputer.transform(df)

Снова проверяем на пропуски. Ниже видно, что пропусков больше нет.

In [ ]:
df[df.columns[pd.isnull(df).sum() > 0]]

In [ ]:
df.tail(10)

Также, можно проверить данные на дубликаты.

In [ ]:
df = df.drop_duplicates()
df.shape

# 6. Визуализация датасета

Сколько фильмов и сериалов от Netflix было выпущено в период с 1980 по 2020 годы 

In [ ]:
plt.figure(figsize=(20,20))
MovTrnd = sns.factorplot("year", data=df, aspect=3, kind="count", color='g')

Какие 10 фильмов собрали самые большие кассовые сборы

In [ ]:
plt.figure(figsize=(10,5))
df4=df.sort_values("gross", ascending=False).head(10)
df_4 = sns.barplot(x="gross", y="name",hue='year', linewidth=0, data=df4,ci= None)
plt.title('10 фильмов с самыми большими кассовыми сборами',fontsize=18,fontweight="bold")
df_4.set_xlim(900000,9990000000) 
plt.legend(bbox_to_anchor=(1.05, 0.9), loc=2, borderaxespad=0.) #
plt.show()

Вывод фильмов в жанре "Хоррор"

In [ ]:
df[df['genre']=='Horror']

Вывод 5 фильмов с актером Jack Nicholson

In [ ]:
df[df['star']=='Jack Nicholson']
df.head(5)